In [ ]:
"""
🎯 COMPLETE DATA CLEANING WORKFLOW
===================================
SaaS Customer & Transaction Data Cleaning Pipeline

Author: Chinyere Obi
Purpose: Clean messy customer and transaction data, upload to Google Sheets
Dataset: Customers (1,500 rows) + Transactions (3,000 rows)
"""

In [1]:
# ============================================================================
# IMPORT LIBRARIES
# ============================================================================

import pandas as pd
import numpy as np
import re
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

# Google Sheets libraries
import gspread
from google.oauth2.service_account import Credentials

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [2]:
# ============================================================================
# LOAD RAW DATA
# ============================================================================

# Load datasets
customers = pd.read_csv(r"C:\Users\hp\Desktop\Dataset\dirty_customers.csv")
transactions = pd.read_csv(r"C:\Users\hp\Desktop\Dataset\dirty_transactions.csv")

print("✅ Datasets loaded successfully!")

✅ Datasets loaded successfully!


In [3]:
# ============================================================================
# 🔍 INITIAL DATA EXPLORATION
# ============================================================================

print("\n" + "="*50)
print("INITIAL DATA QUALITY ASSESSMENT")
print("="*50)

# 1. Table Shapes
print("\n📊 Dataset Dimensions:")
print(f"  • Customers: {customers.shape[0]:,} rows × {customers.shape[1]} columns")
print(f"  • Transactions: {transactions.shape[0]:,} rows × {transactions.shape[1]} columns")

# 2. Column Names
print("\n📋 Column Names:")
print(f"\n  Customers ({len(customers.columns)} columns):")
print(f"  {', '.join(customers.columns.tolist())}")
print(f"\n  Transactions ({len(transactions.columns)} columns):")
print(f"  {', '.join(transactions.columns.tolist())}")

# 3. Data Types
print("\n🔹 Data Types - Customers:")
print(customers.dtypes)
print("\n🔹 Data Types - Transactions:")
print(transactions.dtypes)

# 4. Missing Values Analysis
print("\n⚠️  Missing Values Analysis:")
print("\n  Customers:")
missing_customers = customers.isna().sum()
print(missing_customers[missing_customers > 0])
print(f"  Total missing: {customers.isna().sum().sum():,} cells")

print("\n  Transactions:")
missing_transactions = transactions.isna().sum()
print(missing_transactions[missing_transactions > 0])
print(f"  Total missing: {transactions.isna().sum().sum():,} cells")

# 5. Duplicate Analysis
print("\n🔄 Duplicate Records:")
cust_duplicates = customers.duplicated(subset=['customer_id']).sum()
trans_duplicates = transactions.duplicated(subset=['transaction_id']).sum()
print(f"  • Customers duplicates: {cust_duplicates:,} ({cust_duplicates/len(customers)*100:.1f}%)")
print(f"  • Transactions duplicates: {trans_duplicates:,} ({trans_duplicates/len(transactions)*100:.1f}%)")

# 6. Numeric Summary Statistics
print("\n📈 Numeric Summary - Customers:")
print(customers.describe())

print("\n📈 Numeric Summary - Transactions:")
print(transactions.describe())

# 7. Sample Data Preview
print("\n👀 Data Preview:")
print("\n  First 5 Customers:")
print(customers.head())

print("\n  First 5 Transactions:")
print(transactions.head())


INITIAL DATA QUALITY ASSESSMENT

📊 Dataset Dimensions:
  • Customers: 1,500 rows × 15 columns
  • Transactions: 3,000 rows × 10 columns

📋 Column Names:

  Customers (15 columns):
  customer_id, full_name, email, country, signup_date, subscription_plan, plan_price, payment_method, is_trial, renewal_date, churn_flag, total_logins, last_login_date, lifetime_value, customer_feedback

  Transactions (10 columns):
  transaction_id, customer_id, transaction_date, amount_paid, payment_status, payment_channel, refund_flag, invoice_number, discount_code, support_ticket_id

🔹 Data Types - Customers:
customer_id          object
full_name            object
email                object
country              object
signup_date          object
subscription_plan    object
plan_price           object
payment_method       object
is_trial             object
renewal_date         object
churn_flag           object
total_logins         object
last_login_date      object
lifetime_value       object
customer_f

In [4]:
# ============================================================================
# 💾 STORE RAW COPIES (BEFORE CLEANING)
# ============================================================================

raw_customers = customers.copy()
raw_transactions = transactions.copy()

print("✅ Raw data backed up successfully!")

✅ Raw data backed up successfully!


In [5]:
# ============================================================================
# 🛠️ HELPER FUNCTIONS
# ============================================================================

def _clean_date_helper(date_str):
    """
    Handles multiple date formats and coerces invalid dates to NaT.
    
    Supported formats:
    - YYYY-MM-DD
    - MM/DD/YYYY
    - DD-MM-YYYY
    - Jan 2023, Feb 2024, etc.
    """
    if pd.isna(date_str) or date_str in [None, 'nan', 'NULL', '']:
        return pd.NaT
    
    date_str = str(date_str).strip()
    
    # Handle "Jan 2023", "Feb 2024" format
    if re.match(r'^[A-Za-z]{3}\s\d{4}$', date_str):
        return pd.to_datetime(date_str, format='%b %Y', errors='coerce')
    
    # Handle standard formats
    return pd.to_datetime(date_str, errors='coerce')

print("✅ Helper functions defined!")

✅ Helper functions defined!


In [6]:
# ============================================================================
# 🧼 CLEAN CUSTOMERS TABLE
# ============================================================================

def clean_customers(df):
    """
    Comprehensive cleaning for Customers table.
    
    Steps:
    1. Standardize column names
    2. Clean date columns
    3. Clean numeric columns
    4. Standardize categorical values
    5. Clean boolean fields
    6. Remove duplicates
    """
    
    print("\n" + "="*50)
    print("🧼 CLEANING CUSTOMERS TABLE")
    print("="*50)
    
    original_rows = len(df)
    
    # Step 1: Standardize Column Names
    print("\n1️⃣  Standardizing column names...")
    df.columns = (
        df.columns.str.lower()
        .str.replace(" ", "_")
        .str.replace("_-", "", regex=False)
    )
    print("   ✓ Column names standardized to lowercase_with_underscores")
    
    # Step 2: Clean Date Columns
    print("\n2️⃣  Cleaning date columns...")
    date_columns = ["signup_date", "renewal_date", "last_login_date"]
    for col in date_columns:
        if col in df.columns:
            before_valid = df[col].notna().sum()
            df[col] = df[col].apply(_clean_date_helper)
            after_valid = df[col].notna().sum()
            print(f"   ✓ {col}: {before_valid} → {after_valid} valid dates")
    
    # Step 3: Clean Numeric Columns
    print("\n3️⃣  Cleaning numeric columns...")
    
    # Currency columns
    currency_cols = ["plan_price", "lifetime_value"]
    for col in currency_cols:
        if col in df.columns:
            before_valid = pd.to_numeric(df[col], errors='coerce').notna().sum()
            df[col] = (
                df[col].astype(str)
                .str.replace(r"[^\d.\-]", "", regex=True)  # Remove $, commas
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")
            after_valid = df[col].notna().sum()
            print(f"   ✓ {col}: Removed currency symbols, {before_valid} → {after_valid} valid")
    
    # Total logins
    if "total_logins" in df.columns:
        before_valid = pd.to_numeric(df["total_logins"], errors='coerce').notna().sum()
        df["total_logins"] = (
            df["total_logins"].astype(str)
            .str.replace(" times", "", regex=False)
        )
        df["total_logins"] = pd.to_numeric(df["total_logins"], errors="coerce")
        after_valid = df["total_logins"].notna().sum()
        print(f"   ✓ total_logins: Removed ' times' suffix, {before_valid} → {after_valid} valid")
    
    # Step 4: Standardize Categorical Columns
    print("\n4️⃣  Standardizing categorical values...")
    
    # Country standardization
    country_map = {
        "Can": "Canada", "can ada": "Canada", "Canda": "Canada",
        "De": "Denmark", "Germny": "Germany",
        "india": "India", "INDIA": "India",
        "Nigerai": "Nigeria", "NIgeria": "Nigeria", "Naija": "Nigeria",
        "uk": "United Kingdom", "United Kngdom": "United Kingdom", "U.k": "United Kingdom",
        "U.S": "USA", "U.S.A": "USA", "us": "USA", "Usa": "USA"
    }
    
    if "country" in df.columns:
        before_unique = df["country"].nunique()
        df["country"] = df["country"].astype(str).str.strip().replace(country_map).str.title()
        after_unique = df["country"].nunique()
        print(f"   ✓ country: {before_unique} → {after_unique} unique values")
    
    # Text field cleaning
    if "full_name" in df.columns:
        df["full_name"] = df["full_name"].astype(str).str.title()
        print(f"   ✓ full_name: Converted to Title Case")
    
    if "email" in df.columns:
        df["email"] = df["email"].astype(str).str.strip().str.lower().str.replace(" ", "", regex=False)
        print(f"   ✓ email: Lowercase, removed whitespace")
    
    if "subscription_plan" in df.columns:
        before_unique = df["subscription_plan"].nunique()
        df["subscription_plan"] = df["subscription_plan"].astype(str).str.strip().str.title()
        after_unique = df["subscription_plan"].nunique()
        print(f"   ✓ subscription_plan: {before_unique} → {after_unique} unique values")
        
      # Payment method
    if "payment_method" in df.columns:
        before_unique = df["payment_method"].nunique()
        df["payment_method"] = df["payment_method"].astype(str).str.lower().str.strip().str.title()
        after_unique = df["payment_method"].nunique()
        print(f"   ✓ payment_method: {before_unique} → {after_unique} unique values")
    
    
    # Step 5: Clean Boolean Fields
    print("\n5️⃣  Cleaning boolean fields...")
    
    boolean_map = {
        "TRUE": True, "YES": True, "Y": True, "1": True,
        "FALSE": False, "NO": False, "N": False, "0": False
    }
    
    boolean_cols = ["is_trial", "churn_flag"]
    for col in boolean_cols:
        if col in df.columns:
            before_valid = df[col].notna().sum()
            df[col] = (
                df[col].astype(str)
                .str.strip().str.upper()
                .map(boolean_map)
                .astype("boolean")
            )
            after_valid = df[col].notna().sum()
            print(f"   ✓ {col}: Standardized to boolean, {before_valid} → {after_valid} valid")
    
    # Step 6: Remove Duplicates
    print("\n6️⃣  Removing duplicates...")
    duplicates_removed = df.duplicated(subset=["customer_id"]).sum()
    df.drop_duplicates(subset=["customer_id"], inplace=True)
    print(f"   ✓ Removed {duplicates_removed:,} duplicate customer records")
    
    # Summary
    final_rows = len(df)
    print("\n" + "="*50)
    print(f"✅ CUSTOMERS CLEANING COMPLETE!")
    print(f"   • Original rows: {original_rows:,}")
    print(f"   • Final rows: {final_rows:,}")
    print(f"   • Rows removed: {original_rows - final_rows:,}")
    print("="*50)
    
    return df


# ============================================================================
# 🧽 CLEAN TRANSACTIONS TABLE
# ============================================================================

def clean_transactions(df):
    """
    Comprehensive cleaning for Transactions table.
    
    Steps:
    1. Standardize column names
    2. Clean date column
    3. Clean amount_paid
    4. Standardize categorical fields
    5. Clean boolean fields
    6. Remove duplicates
    """
    
    print("\n" + "="*50)
    print("🧽 CLEANING TRANSACTIONS TABLE")
    print("="*50)
    
    original_rows = len(df)
    
    # Step 1: Standardize Column Names
    print("\n1️⃣  Standardizing column names...")
    df.columns = (
        df.columns.str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("_-", "_", regex=False)
    )
    print("   ✓ Column names standardized")
    
    # Step 2: Clean Date Column
    print("\n2️⃣  Cleaning transaction_date...")
    if "transaction_date" in df.columns:
        before_valid = df["transaction_date"].notna().sum()
        df["transaction_date"] = df["transaction_date"].apply(_clean_date_helper)
        after_valid = df["transaction_date"].notna().sum()
        print(f"   ✓ transaction_date: {before_valid} → {after_valid} valid dates")
    
    # Step 3: Clean Amount Paid
    print("\n3️⃣  Cleaning amount_paid...")
    if "amount_paid" in df.columns:
        before_valid = pd.to_numeric(df["amount_paid"], errors='coerce').notna().sum()
        df["amount_paid"] = (
            df["amount_paid"].astype(str)
            .str.replace(r"[^\d.\-]", "", regex=True)
        )
        df["amount_paid"] = pd.to_numeric(df["amount_paid"], errors="coerce")
        after_valid = df["amount_paid"].notna().sum()
        print(f"   ✓ amount_paid: Removed currency symbols, {before_valid} → {after_valid} valid")
    
    # Step 4: Standardize Categorical Fields
    print("\n4️⃣  Standardizing categorical fields...")
    
    # Payment status
    status_map = {
        "DONE": "Success", "paid": "Success", "Paid": "Success",
        "success": "Success", "FAILED": "Failed"
    }
    
    if "payment_status" in df.columns:
        before_unique = df["payment_status"].nunique()
        df["payment_status"] = df["payment_status"].astype(str).str.strip().replace(status_map).str.title()
        after_unique = df["payment_status"].nunique()
        print(f"   ✓ payment_status: {before_unique} → {after_unique} unique values")
    
    # Payment channel
    if "payment_channel" in df.columns:
        before_unique = df["payment_channel"].nunique()
        df["payment_channel"] = df["payment_channel"].astype(str).str.lower().str.strip().str.title()
        after_unique = df["payment_channel"].nunique()
        print(f"   ✓ payment_channel: {before_unique} → {after_unique} unique values")
    
    # Discount code
    if "discount_code" in df.columns:
        before_valid = df["discount_code"].notna().sum()
        df["discount_code"] = (
            df["discount_code"].astype(str).str.strip()
            .replace({"???": np.nan, "null": np.nan, "": np.nan})
            .apply(lambda x: x.capitalize() if isinstance(x, str) else x)
        )
        after_valid = df["discount_code"].notna().sum()
        print(f"   ✓ discount_code: Cleaned ??? placeholders, {before_valid} → {after_valid} valid")
    
    # Invoice number
    if "invoice_number" in df.columns:
        before_format = (~df["invoice_number"].astype(str).str.startswith("Inv-")).sum()
        df["invoice_number"] = (
            df["invoice_number"].astype(str)
            .str.strip()
            .str.upper()
            .replace({"???": np.nan, "NULL": np.nan, "": np.nan})
            .str.replace(r"[\s_-]*INV[\s_-]*", "Inv-", regex=True)
            .apply(lambda x: "Inv-" + re.sub(r"[^0-9]", "", x) if isinstance(x, str) and x.startswith("Inv-") else x)
        )
        after_format = (~df["invoice_number"].astype(str).str.startswith("Inv-")).sum()
        print(f"   ✓ invoice_number: Standardized to 'Inv-XXXX' format")
    
    # Support ticket ID
    if "support_ticket_id" in df.columns:
        df["support_ticket_id"] = (
            df["support_ticket_id"].astype(str)
            .str.strip()
            .replace({"ticket_error": "Tkt-Error"})
            .str.replace(r"[\s_-]*TKT[\s_-]*", "Tkt-", regex=True)
            .apply(lambda x: "Tkt-" + re.sub(r"[^0-9]", "", x) if x.startswith("Tkt-") else x)
        )
        print(f"   ✓ support_ticket_id: Standardized to 'Tkt-XXXX' format")
    
    # Step 5: Clean Boolean Field
    print("\n5️⃣  Cleaning boolean fields...")
    
    boolean_map_tx = {
        "TRUE": True, "Y": True, "YES": True,
        "FALSE": False, "N": False, "NO": False, "0": False
    }
    
    if "refund_flag" in df.columns:
        before_valid = df["refund_flag"].notna().sum()
        df["refund_flag"] = (
            df["refund_flag"].astype(str)
            .str.strip().str.upper()
            .map(boolean_map_tx)
            .astype("boolean")
        )
        after_valid = df["refund_flag"].notna().sum()
        print(f"   ✓ refund_flag: Standardized to boolean, {before_valid} → {after_valid} valid")
    
    # Step 6: Remove Duplicates
    print("\n6️⃣  Removing duplicates...")
    duplicates_removed = df.duplicated(subset=["transaction_id"]).sum()
    df.drop_duplicates(subset=["transaction_id"], inplace=True)
    print(f"   ✓ Removed {duplicates_removed:,} duplicate transaction records")
    
    # Summary
    final_rows = len(df)
    print("\n" + "="*50)
    print(f"✅ TRANSACTIONS CLEANING COMPLETE!")
    print(f"   • Original rows: {original_rows:,}")
    print(f"   • Final rows: {final_rows:,}")
    print(f"   • Rows removed: {original_rows - final_rows:,}")
    print("="*50)
    
    return df

# ============================================================================
# 🚀 EXECUTE CLEANING
# ============================================================================

start_time = datetime.now()

cleaned_customers = clean_customers(customers)
cleaned_transactions = clean_transactions(transactions)

end_time = datetime.now()
processing_time = (end_time - start_time).total_seconds()

print(f"\n⏱️  Total processing time: {processing_time:.2f} seconds")


🧼 CLEANING CUSTOMERS TABLE

1️⃣  Standardizing column names...
   ✓ Column names standardized to lowercase_with_underscores

2️⃣  Cleaning date columns...
   ✓ signup_date: 1196 → 910 valid dates
   ✓ renewal_date: 768 → 383 valid dates
   ✓ last_login_date: 718 → 358 valid dates

3️⃣  Cleaning numeric columns...
   ✓ plan_price: Removed currency symbols, 1121 → 1500 valid
   ✓ lifetime_value: Removed currency symbols, 321 → 617 valid
   ✓ total_logins: Removed ' times' suffix, 369 → 736 valid

4️⃣  Standardizing categorical values...
   ✓ country: 24 → 9 unique values
   ✓ full_name: Converted to Title Case
   ✓ email: Lowercase, removed whitespace
   ✓ subscription_plan: 12 → 5 unique values
   ✓ payment_method: 5 → 5 unique values

5️⃣  Cleaning boolean fields...
   ✓ is_trial: Standardized to boolean, 1171 → 1171 valid
   ✓ churn_flag: Standardized to boolean, 1280 → 1280 valid

6️⃣  Removing duplicates...
   ✓ Removed 15 duplicate customer records

✅ CUSTOMERS CLEANING COMPLETE!


In [8]:
print("🔍 DATA OVERVIEW AFTER CLEANING")
print("="*50)

print("\n📋 CUSTOMERS TABLE:")
print(customers.dtypes)

print("\n📋 TRANSACTIONS TABLE:")
print(transactions.dtypes)


# Sample Data Preview
print("\n  First 5 Customers:")
print(customers.head())

print("\n  First 5 Transactions:")
print(transactions.head())

🔍 DATA OVERVIEW AFTER CLEANING

📋 CUSTOMERS TABLE:
customer_id                  object
full_name                    object
email                        object
country                      object
signup_date          datetime64[ns]
subscription_plan            object
plan_price                    int64
payment_method               object
is_trial                    boolean
renewal_date         datetime64[ns]
churn_flag                  boolean
total_logins                float64
last_login_date      datetime64[ns]
lifetime_value              float64
customer_feedback            object
dtype: object

📋 TRANSACTIONS TABLE:
transaction_id               object
customer_id                  object
transaction_date     datetime64[ns]
amount_paid                   int64
payment_status               object
payment_channel              object
refund_flag                 boolean
invoice_number               object
discount_code                object
support_ticket_id            object
dtype: objec

In [9]:
# ============================================================================
# ✅ DATA VALIDATION
# ============================================================================

print("\n" + "="*50)
print("✅ POST-CLEANING VALIDATION")
print("="*50)

# 1. Foreign Key Check
print("\n1️⃣  Foreign key integrity check...")
invalid_cust = ~cleaned_transactions["customer_id"].isin(cleaned_customers["customer_id"])
invalid_count = invalid_cust.sum()

if invalid_count > 0:
    print(f"   ⚠️  Found {invalid_count:,} transactions with invalid customer_id")
else:
    print(f"   ✅ All transaction customer_ids are valid!")

# 2. Data Quality Scores
print("\n2️⃣  Data quality scores...")

def calculate_quality_score(df, required_cols):
    """Calculate data quality score based on completeness"""
    total_cells = len(df) * len(required_cols)
    missing_cells = df[required_cols].isna().sum().sum()
    score = ((total_cells - missing_cells) / total_cells) * 100
    return score

# Define critical columns
customers_critical = ['customer_id', 'email', 'country', 'subscription_plan']
transactions_critical = ['transaction_id', 'customer_id', 'amount_paid', 'payment_status']

cust_score = calculate_quality_score(cleaned_customers, customers_critical)
trans_score = calculate_quality_score(cleaned_transactions, transactions_critical)

print(f"   • Customers quality score: {cust_score:.1f}/100")
print(f"   • Transactions quality score: {trans_score:.1f}/100")

# 3. Summary Statistics
print("\n3️⃣  Summary statistics...")
print(f"\n   Customers:")
print(f"   • Total records: {len(cleaned_customers):,}")
print(f"   • Unique customer_ids: {cleaned_customers['customer_id'].nunique():,}")
print(f"   • Missing emails: {cleaned_customers['email'].isna().sum():,}")
print(f"   • Countries: {cleaned_customers['country'].nunique()}")
print(f"   • Subscription plans: {cleaned_customers['subscription_plan'].nunique()}")

print(f"\n   Transactions:")
print(f"   • Total records: {len(cleaned_transactions):,}")
print(f"   • Unique transaction_ids: {cleaned_transactions['transaction_id'].nunique():,}")
print(f"   • Date range: {cleaned_transactions['transaction_date'].min()} to {cleaned_transactions['transaction_date'].max()}")
print(f"   • Total revenue: ${cleaned_transactions['amount_paid'].sum():,.2f}")
print(f"   • Payment channels: {cleaned_transactions['payment_channel'].nunique()}")



✅ POST-CLEANING VALIDATION

1️⃣  Foreign key integrity check...
   ⚠️  Found 77 transactions with invalid customer_id

2️⃣  Data quality scores...
   • Customers quality score: 100.0/100
   • Transactions quality score: 100.0/100

3️⃣  Summary statistics...

   Customers:
   • Total records: 1,485
   • Unique customer_ids: 1,484
   • Missing emails: 0
   • Countries: 9
   • Subscription plans: 5

   Transactions:
   • Total records: 1,571
   • Unique transaction_ids: 1,570
   • Date range: 2020-01-07 00:00:00 to 2025-11-12 00:00:00
   • Total revenue: $309,859.00
   • Payment channels: 5


In [10]:
# ============================================================================
# GENERATE CLEANING LOG
# ============================================================================

print("\n" + "="*50)
print("GENERATING CLEANING LOG")
print("="*50)

summary = {
    "Dataset": ["Customers", "Transactions"],
    "Rows Before": [raw_customers.shape[0], raw_transactions.shape[0]],
    "Rows After": [cleaned_customers.shape[0], cleaned_transactions.shape[0]],
    "Duplicates Removed": [
        raw_customers.duplicated(subset=["customer_id"]).sum(),
        raw_transactions.duplicated(subset=["transaction_id"]).sum()
    ],
    "Invalid Links": [0, invalid_count],
    "Quality Score": [f"{cust_score:.1f}%", f"{trans_score:.1f}%"],
    "Last Cleaned": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] * 2
}

cleaning_log = pd.DataFrame(summary)
print("\n✅ Cleaning Log:")
print(cleaning_log.to_string(index=False))


GENERATING CLEANING LOG

✅ Cleaning Log:
     Dataset  Rows Before  Rows After  Duplicates Removed  Invalid Links Quality Score        Last Cleaned
   Customers         1500        1485                  15              0        100.0% 2025-11-14 22:05:40
Transactions         3000        1571                1429             77        100.0% 2025-11-14 22:05:40


In [11]:
# ============================================================================
# SAVE CLEANED DATA LOCALLY
# ============================================================================

print("\n" + "="*50)
print("SAVING CLEANED DATA")
print("="*50)

# Prepare for CSV export (replace NaT/NaN with empty strings)
cleaned_customers_export = cleaned_customers.astype(object).where(pd.notnull(cleaned_customers), "")
cleaned_transactions_export = cleaned_transactions.astype(object).where(pd.notnull(cleaned_transactions), "")

# Save to CSV
cleaned_customers_export.to_csv("cleaned_customers.csv", index=False)
cleaned_transactions_export.to_csv("cleaned_transactions.csv", index=False)

print("✅ Saved: cleaned_customers.csv")
print("✅ Saved: cleaned_transactions.csv")


SAVING CLEANED DATA
✅ Saved: cleaned_customers.csv
✅ Saved: cleaned_transactions.csv


In [12]:
# ============================================================================
# ☁️ UPLOAD TO GOOGLE SHEETS
# ============================================================================

def upload_to_gsheet(sheet_name, dataframe):
    """
    Upload DataFrame to Google Sheets.
    
    Args:
        sheet_name: Name of the worksheet
        dataframe: DataFrame to upload
    """
    print(f"\n Uploading '{sheet_name}' to Google Sheets...")
    
    try:
        # Google API scopes
        scope = [
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ]
        
        # Authenticate
        creds = Credentials.from_service_account_file("credentials.json", scopes=scope)
        client = gspread.authorize(creds)
        
        # Open target spreadsheet
        spreadsheet = client.open("SaaS Cleaned Data")
        
        # Try to get existing worksheet or create new one
        try:
            ws = spreadsheet.worksheet(sheet_name)
            ws.clear()
            print(f"   ✓ Cleared existing '{sheet_name}' worksheet")
        except gspread.exceptions.WorksheetNotFound:
            ws = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")
            print(f"   ✓ Created new '{sheet_name}' worksheet")
        
        # Convert DataFrame to list of lists
        data_to_upload = [dataframe.columns.values.tolist()] + dataframe.values.tolist()
        
        # Upload data
        ws.update('A1', data_to_upload)
        
        print(f"✅ Successfully uploaded {len(dataframe):,} rows to '{sheet_name}'!")
        print(f"🔗 Sheet URL: {spreadsheet.url}")
        
    except FileNotFoundError:
        print("❌ Error: credentials.json not found!")
        print("   Please ensure your Google service account key is in the same directory.")
    except Exception as e:
        print(f"❌ Error uploading to Google Sheets: {str(e)}")

# Prepare data for Google Sheets (convert all to strings, replace NaT)
print("\n" + "="*50)
print("☁️  UPLOADING TO GOOGLE SHEETS")
print("="*50)

cleaned_customers_gsheet = cleaned_customers.astype(str).replace("NaT", "")
cleaned_transactions_gsheet = cleaned_transactions.astype(str).replace("NaT", "")


# Upload all data
upload_to_gsheet("Cleaned_Customers", cleaned_customers_gsheet)
upload_to_gsheet("Cleaned_Transactions", cleaned_transactions_gsheet)
upload_to_gsheet("Cleaning_Log", cleaning_log)



☁️  UPLOADING TO GOOGLE SHEETS

 Uploading 'Cleaned_Customers' to Google Sheets...
   ✓ Cleared existing 'Cleaned_Customers' worksheet
✅ Successfully uploaded 1,485 rows to 'Cleaned_Customers'!
🔗 Sheet URL: https://docs.google.com/spreadsheets/d/1_Yof_H_kmaivmAYT3TGmMJlqp-lSeBdHD3WJ5EEPQSw

 Uploading 'Cleaned_Transactions' to Google Sheets...
   ✓ Cleared existing 'Cleaned_Transactions' worksheet
✅ Successfully uploaded 1,571 rows to 'Cleaned_Transactions'!
🔗 Sheet URL: https://docs.google.com/spreadsheets/d/1_Yof_H_kmaivmAYT3TGmMJlqp-lSeBdHD3WJ5EEPQSw

 Uploading 'Cleaning_Log' to Google Sheets...
   ✓ Cleared existing 'Cleaning_Log' worksheet
✅ Successfully uploaded 2 rows to 'Cleaning_Log'!
🔗 Sheet URL: https://docs.google.com/spreadsheets/d/1_Yof_H_kmaivmAYT3TGmMJlqp-lSeBdHD3WJ5EEPQSw


In [13]:
# ============================================================================
# 🎉 FINAL SUMMARY
# ============================================================================

print("\n" + "="*50)
print("🎉 DATA CLEANING PIPELINE COMPLETED!")
print("="*50)

print(f"""
📊 FINAL SUMMARY:
  
  Customers:
    • Original: {raw_customers.shape[0]:,} rows
    • Cleaned: {cleaned_customers.shape[0]:,} rows
    • Removed: {raw_customers.shape[0] - cleaned_customers.shape[0]:,} rows
    • Quality: {cust_score:.1f}/100
  
  Transactions:
    • Original: {raw_transactions.shape[0]:,} rows
    • Cleaned: {cleaned_transactions.shape[0]:,} rows
    • Removed: {raw_transactions.shape[0] - cleaned_transactions.shape[0]:,} rows
    • Quality: {trans_score:.1f}/100
  
  Performance:
    • Processing time: {processing_time:.2f} seconds
    • Time saved vs manual: ~7h 59m 58s
  
  Output:
    ✅ CSV files saved locally
    ✅ Data uploaded to Google Sheets
    ✅ Cleaning log generated

""")

print("="*50)
print("✨ All tasks completed successfully!")
print("="*50)


🎉 DATA CLEANING PIPELINE COMPLETED!

📊 FINAL SUMMARY:
  
  Customers:
    • Original: 1,500 rows
    • Cleaned: 1,485 rows
    • Removed: 15 rows
    • Quality: 100.0/100
  
  Transactions:
    • Original: 3,000 rows
    • Cleaned: 1,571 rows
    • Removed: 1,429 rows
    • Quality: 100.0/100
  
  Performance:
    • Processing time: 1.77 seconds
    • Time saved vs manual: ~7h 59m 58s
  
  Output:
    ✅ CSV files saved locally
    ✅ Data uploaded to Google Sheets
    ✅ Cleaning log generated


✨ All tasks completed successfully!
